# Fix pathing

In [1]:
import sys


sys.path.append("..")


In [2]:
import constants

import os


constants.PROJECT_DIRECTORY_PATH = os.path.dirname(os.path.dirname(constants.PROJECT_DIRECTORY_PATH))


# Imports

In [3]:
import datahandler
import utils
import pathing

import networkx
import osmnx as ox
import numpy as np
import pandas as pd


# Constants

In [4]:
data_preprocessor = datahandler.DataPreprocessorOUS_V2()
data_preprocessor.execute()

data_loader = datahandler.DataLoader(datahandler.DataPreprocessorOUS_V2)
data_loader.execute(clean=False, processed=False, enhanced=False)

Enhancing dataset: 100%|██████████| 2/2 [00:00<00:00, 2001.10it/s]


# Methods

In [5]:
def fix_lat_lon(benchmark_times):
    new_benchmark_times = {}

    for ((start_lat, start_lon), (end_lat, end_lon)), benchmark in benchmark_times.items():
        new_benchmark_times[((start_lon, start_lat), (end_lon, end_lat))] = benchmark

    return new_benchmark_times

In [6]:
# times from Google Maps (Sunday, start at 02:00)
benchmark_times = {
    ((59.9369806, 10.7343661), (59.6800045, 10.6572104)): (35, 45),
    ((59.93596, 10.73244), (59.92629, 10.77570)): (4, 9),
    ((59.90864, 10.73921), (59.93037, 10.77273)): (9, 16),
    ((59.92727, 10.73174), (59.86305, 10.66617)): (50, 60),
    ((59.82052, 10.47168), (59.95577, 11.04773)): (40, 50),
    ((60.00352, 10.76216), (59.83905, 10.80742)): (28, 28),

    ((60.31262, 11.14674), (60.11964, 11.47577)): (40, 40),
    ((60.11964, 11.47577), (60.13423, 11.166858)): (22, 26),
    ((60.13423, 11.166858), (59.879044, 11.561687)): (50, 50),
    ((59.879044, 11.561687), (59.932076, 10.987775)): (40, 40),
    ((59.932076, 10.987775), (60.042152, 10.880784)): (20, 22),
    ((60.042152, 10.880784), (59.930733, 10.831094)): (20, 20),
    ((59.930733, 10.831094), (59.917, 10.758744)): (9, 14),
    ((59.917, 10.758744), (59.93917, 10.741926)): (7, 14),
    ((59.93917, 10.741926), (59.71566, 10.853331)): (35, 35),
    ((59.71566, 10.853331), (59.659687, 10.725881)): (18, 18),
    ((59.659687, 10.725881), (59.833073, 10.806946)): (22, 22),
    ((59.833073, 10.806946), (59.830055, 10.439887)): (30, 40),
    ((59.830055, 10.439887), (59.89813, 10.509723)): (14, 14),
    ((59.89813, 10.509723), (59.939663, 10.687367)): (16, 20),
    ((59.939663, 10.687367), (59.893173, 10.806364)): (12, 16),
    ((59.893173, 10.806364), (59.960304, 10.884091)): (12, 18),
    ((59.960304, 10.884091), (59.997875, 11.03928)): (20, 20),
    ((59.997875, 11.03928), (59.917873, 10.585751)): (28, 35),
    ((59.917873, 10.585751), (60.31262, 11.14674)): (55, 65),
}

benchmark_times = fix_lat_lon(benchmark_times)

In [7]:
def get_node(od: pathing.OriginDestination, x, y):
    if (x, y) in od.node_cache:
        return od.node_cache[(x, y)]
    
    while True:
        node = ox.distance.nearest_nodes(od.graph, x, y)
        if networkx.has_path(od.graph, node, od.node_validator) and networkx.has_path(od.graph, od.node_validator, node):
            od.node_cache[(x, y)] = node
            return node
        else:
            od.graph.remove_node(node)


In [23]:
def calculate_error(od: pathing.OriginDestination, benchmark_times):
    calculated_times = {}

    for ((start_lon, start_lat), (end_lon, end_lat)), _ in benchmark_times.items():
        # Convert geographic coordinates to UTM and get the corresponding node
        start_x, start_y = utils.geographic_to_utm(start_lon, start_lat, od.utm_epsg)
        end_x, end_y = utils.geographic_to_utm(end_lon, end_lat, od.utm_epsg)

        start_node = od.get_node_from_lat_lon(start_lat, start_lon)
        end_node = od.get_node_from_lat_lon(end_lat, end_lon)

        if start_node is None or end_node is None:
            continue

        # Calculate the shortest path and total travel time
        shortest_time_path = ox.shortest_path(
            od.graph,
            start_node,
            end_node,
            weight='time'
        )
        total_travel_time = sum(od.graph[u][v][0]['time'] for u, v in zip(shortest_time_path[:-1], shortest_time_path[1:]))

        # Store the calculated travel time
        calculated_times[((start_lon, start_lat), (end_lon, end_lat))] = total_travel_time

        # Print the calculated travel time and compare with the benchmark
        benchmark_range = benchmark_times[((start_lon, start_lat), (end_lon, end_lat))]
        print(f"Path: ({start_lon}, {start_lat}) to ({end_lon}, {end_lat})")
        print(f"Calculated Travel Time: {total_travel_time:.2f} minutes")
        print(f"Benchmark Travel Time Range: {benchmark_range[0]} - {benchmark_range[1]} minutes")
        print()

    return calculated_times

# Main

In [25]:
od = pathing.OriginDestination(
    dataset_id="oslo",
    utm_epsg=f"EPSG:326{33}"
)

od.get_graph()

In [29]:
calculated_times = calculate_error(od, benchmark_times)


Mean Squared Error: 915142.2846161921
Coordinates: ((10.7343661, 59.9369806), (10.6572104, 59.6800045)), Calculated Time: 1203.3897666666667
Coordinates: ((10.73244, 59.93596), (10.7757, 59.92629)), Calculated Time: 94.96826666666666
Coordinates: ((10.73921, 59.90864), (10.77273, 59.93037)), Calculated Time: 137.63216666666668
Coordinates: ((10.73174, 59.92727), (10.66617, 59.86305)), Calculated Time: 1572.6406333333337
Coordinates: ((10.47168, 59.82052), (11.04773, 59.95577)), Calculated Time: 1484.3633333333335
Coordinates: ((10.76216, 60.00352), (10.80742, 59.83905)), Calculated Time: 705.9755333333335
Coordinates: ((11.14674, 60.31262), (11.47577, 60.11964)), Calculated Time: 1141.6837666666663
Coordinates: ((11.47577, 60.11964), (11.166858, 60.13423)), Calculated Time: 682.8854999999999
Coordinates: ((11.166858, 60.13423), (11.561687, 59.879044)), Calculated Time: 1643.3071666666674
Coordinates: ((11.561687, 59.879044), (10.987775, 59.932076)), Calculated Time: 1381.1309333333338


In [28]:
def verify_and_set_edge_attributes(graph):
    for u, v, data in graph.edges(data=True):
        if 'time' not in data:
            # Set a default time or calculate it based on other attributes, like length or speed limit
            # This is a simplified example; you should adjust it based on your actual data
            data['time'] = data.get('length', 0) / 30  # Example: default speed = 30 units

# Call this function with your graph instance before running the calculate_error function
verify_and_set_edge_attributes(od.graph)


In [31]:
def print_calculated_vs_benchmark_times(od, benchmark_times):
    calculated_times = {}

    for ((start_lat, start_lon), (end_lat, end_lon)), _ in benchmark_times.items():
        # Convert geographic coordinates to UTM and get the corresponding node
        start_x, start_y = utils.geographic_to_utm(start_lon, start_lat, od.utm_epsg)
        end_x, end_y = utils.geographic_to_utm(end_lon, end_lat, od.utm_epsg)

        start_node = get_node(od, start_x, start_y)
        end_node = get_node(od, end_x, end_y)

        # Calculate the shortest path and total travel time
        shortest_path = od.shortest_path(
            od.graph,
            start_node,
            end_node,
            weight='time'
        )
        total_travel_time = sum(od.graph[u][v][0]['time'] for u, v in zip(shortest_path[:-1], shortest_path[1:]))

        # Store the calculated travel time
        calculated_times[((start_lat, start_lon), (end_lat, end_lon))] = total_travel_time

    # Print calculated times vs benchmark times
    for path, (min_time, max_time) in benchmark_times.items():
        calc_time = calculated_times[path]
        print(f"Coordinates: {path}, Calculated Ambulance Time: {calc_time:.2f} minutes, Benchmark Time Range: ({min_time}, {max_time}) minutes")

# Assuming benchmark_times is defined as shown previously and od is your OriginDestination instance
print_calculated_vs_benchmark_times(od, benchmark_times)


TypeError: '<=' not supported between instances of 'int' and 'str'